In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



# CAPSTONE PROJECT - Week 2

### Applied Data Science Capstone by IBM/Coursera

### Table of Contents

* [Introduction](#intro)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
    * [Geospatial Data](#geo)
    * [Demographics](#demo)
* [Discussion](#discussion)    
* [Conclusion](#conclusion)

## Introduction<a class="anchor" id="intro"></a>

## Investment opportunity in Regina, Saskatchewan, Canada.

#### Regina is the capital city of the province of Saskatchewan, in Canada. In the heart of the prairies, Regina is a multicultural hub in the province, and it has room for venues with different ethnicity roots to prosper. 

The motivation behind this study is to find potential investment opportunities in Regina. Using different data sources, this study aims to combine different perspectives in regards to where to establish a new venture and what kind of venture an entrepreneur could invest when thinking of opening a business in Regina.

##### Target Audience:
This study is targeted to entrepreneurs, venture capitalist or investors looking for business opportunities.

##### Question problem:
What kind of venue and in which area of the city would a business most likely succeed in Regina?

## Data<a class="anchor" id="data"></a>

### In this section I will describe where different data will be collected for the completion of this assignment

This study will collect two main fields of data:

**Geospatial data**, 

which focuses on getting the geospatial coordinates, city areas postal codes and info about venues around each area.

**Demographics data**,

which focuses in understanding more about the population of Regina and how it is distributed around the city.

#### Geospatial Data
To analyze the geospatial data from Regina, including its venues I will rely on:

https://www.geonames.org/postalcode-search.html?q=regina&country=CA&adminCode1=SK - To provide the available Postal Codes and Area Names of the city

https://www.google.com - to collect the exact coordinates from each of the previous collected postal codes.

And then, using **FourSquare API Data** I will use the above mentioned coordinates to collect data from different venues located around the city, using a range of 700 to provide a broader idea of each area of the city. 

#### Demographics

I will collect the more info about the demographics of Regina, using the **Census of 2016**, provided by Stats Canada. 

Available on:https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=POPC&Code1=0698&Geo2=PR&Code2=47&SearchText=Regina&SearchType=Begins&SearchPR=01&B1=All&GeoLevel=PR&GeoCode=0698&TABID=1&type=0

And also the **website of the City of Regina**, which provides a detailed profile from areas of the city. 

Available on:https://www.regina.ca/about-regina/neighbourhood-profiles/

## Methodology <a class="anchor" id="methodology"></a>

In this project we will focus our efforts in trying to find good opportunities in the city of Regina for entrepreneurs to invest in a new venture. 

To do so we will split this study in seven steps:
* First we will collect geospatial data about the city in order to be able to identify the different areas of the city
* Second we will use the geolocator and folium to build Regina's map and pinpoint the different areas in it
* Third we will use the Foursquare API in order to locate the different venues located around the city, with a radius of 700 around each pinpointed area
* Fourth we will separate the areas into clusters based on their venues similarities
* Fifth we will analyze the demographics of the city, using 2016 census 
* Sixth we will analyze the neighbourhoods profile to spot where there is a potential for a new venture
* Seventh we will combine the information acquired and draw a conclusion

## Analysis <a class="anchor" id="analysis"></a>

The analysis section is where we will conduct all the study and analyze the results.

As described in the methodology, we will first start with the Geospatial Data and then move to Demographics

### Geospatial Data<a class="anchor" id="geo"></a>

My first step is to collect **geospatial data** from Regina, Saskastchewan. 

First, I will collect the postal code and area names from Regina from:https://www.geonames.org/postalcode-search.html?q=regina&country=CA&adminCode1=SK. This website also contains geospatial coordinates, however they are not accurate.
Google provided a better picture of the accurate coordinates for each postal code, and for this reason I combined the data from both sources in one csv spreadsheet which can be found in my github: https://raw.githubusercontent.com/jptxmelo/Capstone_FinalProject/main/reginaarea.csv

In [2]:
postal = pd.read_csv("https://raw.githubusercontent.com/jptxmelo/Capstone_FinalProject/main/reginaarea.csv")

postal.head()

,Postal Code,Area,Latitude,Longitude
0,S4V,Regina Southeast,50.418,-104.530
1,S4W,Regina Southwest,50.434,-104.660
2,S4X,Regina Northwest,50.506,-104.674
3,S4Z,Regina Northeast,50.452,-104.531
4,S4L,Regina East,50.434,-104.350


Using geolocator we will get the data required so folium can build the map, then using the data from the table above we will pinpoint each different area of the city

In [3]:
address = 'Regina, SK, CA'

geolocator = Nominatim(user_agent="Regina_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Regina are {}, {}.'.format(latitude, longitude))

# create map of Regina using latitude and longitude values
map_regina = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(postal['Latitude'], postal['Longitude'], postal['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_regina)  
    
map_regina

The geograpical coordinate of Regina are 50.44876, -104.61731.


Using foursquare API I will collect the information from the venues around each neighborhood, and then using getNearbyVenues function I will be able to retrieve the same info for all remaining neighborhoods.

In [4]:
CLIENT_ID = 'SPYWFEUK4P2X2ICJIYKBRKHNGNZX5IET435FQVFIIOOS5BDA' # your Foursquare ID
CLIENT_SECRET = '2IFPTOWMHL43K1BRFSGLYXOWKKIHNUHF5BCAQ2UQU4NJ3FJ1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [5]:
postal.loc[0, 'Area']
area_latitude = postal.loc[0, 'Latitude'] # neighborhood latitude value
area_longitude = postal.loc[0, 'Longitude'] # neighborhood longitude value

area_name = postal.loc[0, 'Area'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(area_name, 
                                                               area_latitude, 
                                                               area_longitude))

Latitude and longitude values of Regina Southeast are 50.418, -104.53.


Because of the low number of results when running on the first time with radius = 500, I decided to increase it to 700

In [6]:
LIMIT = 100
radius = 700
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    area_latitude, 
    area_longitude, 
    radius, 
    LIMIT)

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60a07f768e03f54a67dc4fc1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Regina',
  'headerFullLocation': 'Regina',
  'headerLocationGranularity': 'city',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 50.42430000630001,
    'lng': -104.52013116423677},
   'sw': {'lat': 50.41169999369999, 'lng': -104.53986883576323}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d876bfc90dca0906d7b9c3d',
       'name': 'Rockstar Lounge (rockstarhomes.ca)',
       'location': {'address': '3889 Arcola Ave E',
        'crossStreet': 'Wascana View Dr',
        'lat': 50.419018902737065,
        'lng': -104.52735900

Unfortunatelly, as expressed by the code above: There are not a lot of results near this location, even after increasing the radius. However, we want to proceed with the plan.

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-9-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rockstar Lounge (rockstarhomes.ca),Rock Club,50.419019,-104.527359
1,Wascana View Park,Park,50.418163,-104.535159


In [10]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
regina_venues = getNearbyVenues(names=postal['Area'],
                                   latitudes=postal['Latitude'],
                                   longitudes=postal['Longitude']
                                  )

Regina Southeast
Regina Southwest
Regina Northwest
Regina Northeast
Regina East
Regina Central
Regina West
Regina Outer Northwest
Regina Northeast and East Central
Regina North Central
Regina South Saskatchewan Provincial Government
Regina Rural West


In [13]:
print(regina_venues.shape)
regina_venues.head()

(121, 7)


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regina Southeast,50.418,-104.53,Rockstar Lounge (rockstarhomes.ca),50.419019,-104.527359,Rock Club
1,Regina Southeast,50.418,-104.53,Wascana View Park,50.418163,-104.535159,Park
2,Regina Southwest,50.434,-104.66,Tim Hortons,50.433593,-104.655899,Coffee Shop
3,Regina Southwest,50.434,-104.66,Tim Hortons,50.433453,-104.655844,Coffee Shop
4,Regina Southwest,50.434,-104.66,Maple Leaf Lounge,50.433475,-104.655730,Airport Lounge


In [14]:
regina_venues.groupby('Area').count()

,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area,,,,,,
Regina Central,51,51,51,51,51,51
Regina North Central,3,3,3,3,3,3
Regina Northeast,23,23,23,23,23,23
Regina Northeast and East Central,1,1,1,1,1,1
Regina Northwest,2,2,2,2,2,2
Regina Outer Northwest,2,2,2,2,2,2
Regina South Saskatchewan Provincial Government,26,26,26,26,26,26
Regina Southeast,2,2,2,2,2,2
Regina Southwest,6,6,6,6,6,6


In a nutshell we can see that there are not a lot of venues spread around Regina registered on Foursquare API, but we shall proceed 

In [15]:
print('There are {} uniques categories.'.format(len(regina_venues['Venue Category'].unique())))

There are 63 uniques categories.


In [16]:
# one hot encoding
regina_onehot = pd.get_dummies(regina_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
regina_onehot['Area'] = regina_venues['Area'] 

first_column = regina_onehot.pop('Area')
regina_onehot.insert(0, 'Area', first_column)
regina_onehot.head()

,Area,Airport Lounge,American Restaurant,Amphitheater,Asian Restaurant,Bakery,Bank,Baseball Field,Beer Bar,Boutique,Breakfast Spot,Café,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Diner,Discount Store,Donut Shop,English Restaurant,Fast Food Restaurant,Food,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Grocery Store,Gym,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Inn,Irish Pub,Italian Restaurant,Liquor Store,Lounge,Mattress Store,Mexican Restaurant,Miscellaneous Shop,Museum,Park,Pet Store,Pharmacy,Pizza Place,Playground,Poutine Place,Pub,Rental Car Location,Restaurant,Rock Club,Sandwich Place,Shopping Mall,Skating Rink,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Wine Bar
0,Regina Southeast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,Regina Southeast,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Regina Southwest,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Regina Southwest,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Regina Southwest,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
regina_grouped = regina_onehot.groupby('Area').mean().reset_index()
regina_grouped

,Area,Airport Lounge,American Restaurant,Amphitheater,Asian Restaurant,Bakery,Bank,Baseball Field,Beer Bar,Boutique,Breakfast Spot,Café,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Diner,Discount Store,Donut Shop,English Restaurant,Fast Food Restaurant,Food,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Grocery Store,Gym,History Museum,Hookah Bar,Hotel,Ice Cream Shop,Inn,Irish Pub,Italian Restaurant,Liquor Store,Lounge,Mattress Store,Mexican Restaurant,Miscellaneous Shop,Museum,Park,Pet Store,Pharmacy,Pizza Place,Playground,Poutine Place,Pub,Rental Car Location,Restaurant,Rock Club,Sandwich Place,Shopping Mall,Skating Rink,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Wine Bar
0,Regina Central,0.000000,0.019608,0.0,0.039216,0.019608,0.019608,0.0,0.019608,0.019608,0.019608,0.039216,0.098039,0.000000,0.019608,0.000000,0.019608,0.019608,0.000000,0.000000,0.000000,0.000000,0.0,0.019608,0.0,0.000000,0.019608,0.039216,0.000000,0.000000,0.019608,0.019608,0.019608,0.058824,0.000000,0.000000,0.019608,0.019608,0.019608,0.019608,0.000000,0.000000,0.000000,0.0,0.019608,0.019608,0.019608,0.019608,0.0,0.0,0.039216,0.019608,0.039216,0.0,0.039216,0.019608,0.000000,0.000000,0.039216,0.000000,0.000000,0.058824,0.019608,0.019608
1,Regina North Central,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000
2,Regina Northeast,0.000000,0.043478,0.0,0.043478,0.000000,0.043478,0.0,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,0.086957,0.0,0.000000,0.0,0.043478,0.000000,0.043478,0.043478,0.043478,0.000000,0.000000,0.000000,0.130435,0.000000,0.086957,0.000000,0.043478,0.000000,0.000000,0.043478,0.043478,0.000000,0.0,0.000000,0.000000,0.000000,0.043478,0.0,0.0,0.043478,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000
3,Regina Northeast and East Central,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Regina Northwest,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Regina Outer Northwest,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,

Now I will collect what are the top 5 most frequent venues in each neighborhood

In [18]:
num_top_venues = 5

for hood in regina_grouped['Area']:
    print("----"+hood+"----")
    temp = regina_grouped[regina_grouped['Area'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Regina Central----
              venue  freq
0       Coffee Shop  0.10
1   Thai Restaurant  0.06
2             Hotel  0.06
3               Pub  0.04
4  Asian Restaurant  0.04


----Regina North Central----
            venue  freq
0  Discount Store  0.33
1     Supermarket  0.33
2     Gas Station  0.33
3  Airport Lounge  0.00
4       Irish Pub  0.00


----Regina Northeast----
                    venue  freq
0                   Hotel  0.13
1    Fast Food Restaurant  0.09
2                     Inn  0.09
3          Mattress Store  0.04
4  Furniture / Home Store  0.04


----Regina Northeast and East Central----
            venue  freq
0            Park   1.0
1  Airport Lounge   0.0
2      Playground   0.0
3             Inn   0.0
4       Irish Pub   0.0


----Regina Northwest----
            venue  freq
0      Playground   0.5
1            Park   0.5
2  Airport Lounge   0.0
3             Inn   0.0
4       Irish Pub   0.0


----Regina Outer Northwest----
            venue  freq
0  History 

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now I will collect the 10 most common venues for each neighborhood and adding them to a table that will further be used to create clusters that will combine the neighborhoods based on the similiarities they have among their top 10 venues 

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
area_venues_sorted = pd.DataFrame(columns=columns)
area_venues_sorted['Area'] = regina_grouped['Area']

for ind in np.arange(regina_grouped.shape[0]):
    area_venues_sorted.iloc[ind, 1:] = return_most_common_venues(regina_grouped.iloc[ind, :], num_top_venues)

area_venues_sorted

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regina Central,Coffee Shop,Hotel,Thai Restaurant,Sandwich Place,Gastropub,Café,Restaurant,Pub,Steakhouse,Asian Restaurant
1,Regina North Central,Supermarket,Gas Station,Discount Store,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint
2,Regina Northeast,Hotel,Inn,Fast Food Restaurant,Pub,Mexican Restaurant,Donut Shop,Italian Restaurant,Furniture / Home Store,Pizza Place,Coffee Shop
3,Regina Northeast and East Central,Park,Wine Bar,History Museum,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
4,Regina Northwest,Park,Playground,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
5,Regina Outer Northwest,Food,History Museum,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint
6,Regina South Saskatchewan Provincial Government,Coffee Shop,Fast Food Restaurant,Hotel,Restaurant,Pub,Pizza Place,Liquor Store,Convenience Store,Construction & Landscaping,Miscellaneous Shop
7,Regina Southeast,Rock Club,Park,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
8,Regina Southwest,Rental Car Location,Coffee Shop,Airport Lounge,Food,Discount Store,Donut Shop,English Restaurant,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega
9,Regina West,Fried Chicken Joint,Museum,Amphitheater,Poutine Place,Baseball Field,Food,Donut Shop,English Restaurant,Fast Food Restaurant,Wine Bar


In total, we will create 5 different Clusters

In [21]:
# set number of clusters
kclusters = 5

regina_grouped_clustering = regina_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(regina_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 3, 0, 0, 4, 3, 0, 2, 3])

In [22]:
# add clustering labels
area_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

regina_merged = postal

# merge regina_grouped with postal to add latitude/longitude for each neighborhood
regina_merged = regina_merged.join(area_venues_sorted.set_index('Area'), on='Area')

regina_merged

,Postal Code,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,S4V,Regina Southeast,50.418,-104.530,0.0,Rock Club,Park,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
1,S4W,Regina Southwest,50.434,-104.660,2.0,Rental Car Location,Coffee Shop,Airport Lounge,Food,Discount Store,Donut Shop,English Restaurant,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega
2,S4X,Regina Northwest,50.506,-104.674,0.0,Park,Playground,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
3,S4Z,Regina Northeast,50.452,-104.531,3.0,Hotel,Inn,Fast Food Restaurant,Pub,Mexican Restaurant,Donut Shop,Italian Restaurant,Furniture / Home Store,Pizza Place,Coffee Shop
4,S4L,Regina East,50.434,-104.350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,S4P,Regina Central,50.444,-104.610,3.0,Coffee Shop,Hotel,Thai Restaurant,Sandwich Place,Gastropub,Café,Restaurant,Pub,Steakhouse,Asian Restaurant
6,S4T,Regina West,50.455,-104.661,3.0,Fried Chicken Joint,Museum,Amphitheater,Poutine Place,Baseball Field,Food,Donut Shop,English Restaurant,Fast Food Restaurant,Wine Bar
7,S4Y,Regina Outer Northwest,50.475,-104.699,4.0,Food,History Museum,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint
8,S4N,Regina Northeast and East Central,50.464,-104.550,0.0,Park,Wine Bar,History Museum,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
9,S4R,Regina North Central,50.480,-104.630,1.0,Supermarket,Gas Station,Discount Store,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint


Now we have to delete the NaN rows, because they do not present relevant data to proceed with the study.

In [23]:
regina_merged = regina_merged.dropna()
regina_merged.head()

,Postal Code,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,S4V,Regina Southeast,50.418,-104.530,0.0,Rock Club,Park,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
1,S4W,Regina Southwest,50.434,-104.660,2.0,Rental Car Location,Coffee Shop,Airport Lounge,Food,Discount Store,Donut Shop,English Restaurant,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega
2,S4X,Regina Northwest,50.506,-104.674,0.0,Park,Playground,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
3,S4Z,Regina Northeast,50.452,-104.531,3.0,Hotel,Inn,Fast Food Restaurant,Pub,Mexican Restaurant,Donut Shop,Italian Restaurant,Furniture / Home Store,Pizza Place,Coffee Shop
5,S4P,Regina Central,50.444,-104.610,3.0,Coffee Shop,Hotel,Thai Restaurant,Sandwich Place,Gastropub,Café,Restaurant,Pub,Steakhouse,Asian Restaurant


Now we have to convert the Cluster Labels into integers so that we can print the map

In [24]:
regina_merged.dtypes

Postal Code                object
Area                       object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [25]:
regina_merged['Cluster Labels'] = regina_merged['Cluster Labels'].astype(int)

<ipython-input-25-ee824ab8e9be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regina_merged['Cluster Labels'] = regina_merged['Cluster Labels'].astype(int)


In [26]:
regina_merged

,Postal Code,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,S4V,Regina Southeast,50.418,-104.530,0,Rock Club,Park,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
1,S4W,Regina Southwest,50.434,-104.660,2,Rental Car Location,Coffee Shop,Airport Lounge,Food,Discount Store,Donut Shop,English Restaurant,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega
2,S4X,Regina Northwest,50.506,-104.674,0,Park,Playground,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
3,S4Z,Regina Northeast,50.452,-104.531,3,Hotel,Inn,Fast Food Restaurant,Pub,Mexican Restaurant,Donut Shop,Italian Restaurant,Furniture / Home Store,Pizza Place,Coffee Shop
5,S4P,Regina Central,50.444,-104.610,3,Coffee Shop,Hotel,Thai Restaurant,Sandwich Place,Gastropub,Café,Restaurant,Pub,Steakhouse,Asian Restaurant
6,S4T,Regina West,50.455,-104.661,3,Fried Chicken Joint,Museum,Amphitheater,Poutine Place,Baseball Field,Food,Donut Shop,English Restaurant,Fast Food Restaurant,Wine Bar
7,S4Y,Regina Outer Northwest,50.475,-104.699,4,Food,History Museum,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint
8,S4N,Regina Northeast and East Central,50.464,-104.550,0,Park,Wine Bar,History Museum,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
9,S4R,Regina North Central,50.480,-104.630,1,Supermarket,Gas Station,Discount Store,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint
10,S4S,Regina South Saskatchewan Provincial Government,50.413,-104.610,3,Coffee Shop,Fast Food Restaurant,Hotel,Restaurant,Pub,Pizza Place,Liquor Store,Convenience Store,Construction & Landscaping,Miscellaneous Shop


Finally, a map is created to show the different clusters. Each different color in the map represent a different cluster. 

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(regina_merged['Latitude'], regina_merged['Longitude'], regina_merged['Area'], regina_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Details about cluster 0** 

Cluster 0 has only Parks registered as their venues, the other most common venues columns all have 0.0 frequency

In [28]:
regina_merged.loc[regina_merged['Cluster Labels'] == 0, regina_merged.columns[[1] + list(range(5, regina_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Regina Southeast,Rock Club,Park,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
2,Regina Northwest,Park,Playground,Wine Bar,Deli / Bodega,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
8,Regina Northeast and East Central,Park,Wine Bar,History Museum,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop


**Details about cluster 1** 

As one can see, the most common venues in these neighborhoods are related to food venues and hospitality venues

In [29]:
regina_merged.loc[regina_merged['Cluster Labels'] == 1, regina_merged.columns[[1] + list(range(5, regina_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Regina North Central,Supermarket,Gas Station,Discount Store,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Furniture / Home Store,Fried Chicken Joint


**Details about cluster 2** 

Cluster 2 is a residential area, in which the 3 most common venues are related to basic residential area services

In [30]:
regina_merged.loc[regina_merged['Cluster Labels'] == 2, regina_merged.columns[[1] + list(range(5, regina_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Regina Southwest,Rental Car Location,Coffee Shop,Airport Lounge,Food,Discount Store,Donut Shop,English Restaurant,Fast Food Restaurant,Food & Drink Shop,Deli / Bodega


**Details about cluster 3**

In [31]:
regina_merged.loc[regina_merged['Cluster Labels'] == 3, regina_merged.columns[[1] + list(range(5, regina_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Regina Northeast,Hotel,Inn,Fast Food Restaurant,Pub,Mexican Restaurant,Donut Shop,Italian Restaurant,Furniture / Home Store,Pizza Place,Coffee Shop
5,Regina Central,Coffee Shop,Hotel,Thai Restaurant,Sandwich Place,Gastropub,Café,Restaurant,Pub,Steakhouse,Asian Restaurant
6,Regina West,Fried Chicken Joint,Museum,Amphitheater,Poutine Place,Baseball Field,Food,Donut Shop,English Restaurant,Fast Food Restaurant,Wine Bar
10,Regina South Saskatchewan Provincial Government,Coffee Shop,Fast Food Restaurant,Hotel,Restaurant,Pub,Pizza Place,Liquor Store,Convenience Store,Construction & Landscaping,Miscellaneous Shop


**Details about cluster 4**

In [32]:
regina_merged.loc[regina_merged['Cluster Labels'] == 4, regina_merged.columns[[1] + list(range(5, regina_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Regina Outer Northwest,Food,History Museum,Wine Bar,Deli / Bodega,Grocery Store,Gift Shop,Gastropub,Gas Station,Furniture / Home Store,Fried Chicken Joint


#### From the looks of it, the best place to set a new venue, especially if related to food or hospitality, would be in one of the points in Cluster 1

The next step now is to try to understand a little bit more about Regina demographics and try to make a sense of what could be a good investment opportunity in this city

### Demographics<a class="anchor" id="demo"></a>

As described in the data session, to understand better the demographics of Regina, we are using the census 2016 data to get a better understanding of how the population of Regina is composed.

To do that, I download the csv file from StatsCanada and deleted a few empty columns to facilitate the processing.

In [40]:
demographics = pd.read_csv("https://raw.githubusercontent.com/jptxmelo/Capstone_FinalProject/main/CensusProfile2016-ProfilRecensement2016-20210514022451.csv", encoding= 'unicode_escape')

demographics.head()

,Topic,Characteristics,Total
0,Population and dwellings,Population; 2016,214631.0
1,Population and dwellings,Population; 2011,192079.0
2,Population and dwellings,Population percentage change; 2011 to 2016,11.7
3,Population and dwellings,Total private dwellings,92841.0
4,Population and dwellings,Private dwellings occupied by usual residents,87168.0


In [41]:
immigration = demographics[(demographics.Topic == 'Immigrants by selected place of birth')]
immigration.head()

,Topic,Characteristics,Total
1156,Immigrants by selected place of birth,Total - Selected places of birth for the immig...,36115.0
1157,Immigrants by selected place of birth,Americas,2745.0
1158,Immigrants by selected place of birth,Brazil,30.0
1159,Immigrants by selected place of birth,Colombia,50.0
1160,Immigrants by selected place of birth,El Salvador,170.0


In [49]:
immigration.Total = immigration.Total.astype(int)

C:\Users\joao-\anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [50]:
immigration.dtypes

Topic              object
Characteristics    object
Total               int32
dtype: object

In [53]:
immigration.sort_values('Total', ascending=False).head()

,Topic,Characteristics,Total
1156,Immigrants by selected place of birth,Total - Selected places of birth for the immig...,36115
1197,Immigrants by selected place of birth,Asia,23050
1209,Immigrants by selected place of birth,Philippines,7280
1169,Immigrants by selected place of birth,Europe,6055
1202,Immigrants by selected place of birth,India,4255


## Discussion <a class="anchor" id="discussion"></a>

We can see from the data exctracted from the census 2016, that the population of Regina is 214,631 and among those, 36,115 are immigrants. Which represents roughly 17% of the population.

Among the immigrants, Asians count for roughly 2/3 of them, and Philippinos are the biggest group within the Asian Immigrants.

**FOURSQUARE API** did not find any results for venues that had Filipino in their description. For that reason, after searching on Google maps, I found a few specialty food stores and only two Filipino restaurants in the city. 

The stores seem to be widely spread across the city, however only two restaurants seem to leave a gap in other areas of the city.

The two restaurants are:

**SA Favorito Foods** - a small local restaurant in the North Central Area, cluster 2

**Jollibee** - a famous fastfood chain located in the Northeast Area, cluster 1

With that said, we can look now into the neighbourhood profiles that were framed by the City of Regina, in order to find what could be a good neighbourhood to place a new Filipino restaurant.

https://www.regina.ca/about-regina/neighbourhood-profiles/
After careful examination of each one of the neighbourhoods profiles among cluster 1, there was one neighbourhood that stand out among the others for a few interesting points.

The Harbour Landing neighbourhood has had the highest increase in immigration since 2011, its current population is composed 34% of immigrants of whom about 70% are Asian. 
Harbour Landing also holds an average household income higher than most of other neighbourhoods.

With that said, it looks like Harbour Landing could offer a good potential for a higher-end Filipino Restaurant, which currently is non-existent in the city.

## CONCLUSION <a class="anchor" id="conclusion"></a>

After examining the areas of the city, the demographics we come to the conclusion that there is a good opportunity for entrepreneurs to invest in a high-end Filipino Restaurant in the Southwest area of the city, more especifically in Harbour Landing. 